# Import, filter data

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [ ]:
# Import dataset
raw_data_df = pd.read_csv("data-compas/compas-scores-two-years.csv")
raw_data_df.shape

In [ ]:
raw_data_df[:5]

In [ ]:
# Filter out the dataset similarly to the way ProPublica did
# see: https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb
filter_data_df = raw_data_df
filter_data_df = filter_data_df[filter_data_df['days_b_screening_arrest'] >= -30]
filter_data_df = filter_data_df[filter_data_df['days_b_screening_arrest'] <= 30]
filter_data_df = filter_data_df[filter_data_df['is_recid'] != -1]
filter_data_df = filter_data_df[filter_data_df['c_charge_degree'] != 'O']
filter_data_df = filter_data_df[filter_data_df['score_text'] != 'N/A']
filter_data_df.shape

## Split data into "training", "test"

In [ ]:
split_ratio = 0.8

filter_data_df_src = filter_data_df.sample(n=int(split_ratio*len(filter_data_df)))
filter_data_df_test = filter_data_df[~filter_data_df.index.isin(filter_data_df_src.index)]

print(filter_data_df_src.shape)
print(filter_data_df_test.shape)

# Define rationalization

In [ ]:
# Fields accessible to use in rationalization?
# ... some of which are VERY unfair
accessible_fields = [
    # 'sex', # >:(
    # 'race', # >:(
    'age',
    'age_cat',
    'juv_fel_count',
    'decile_score',
    'juv_misd_count',
    'juv_other_count',
    'priors_count',
    'c_charge_degree',
    'c_charge_desc',
]
# Field we are trying to rationalize why it could be 1 vs 0
justifying_field = 'is_recid' # 'is_recid' or 'is_violent_recid'
# Field that is the actual data for what we are trying to determine
trueresult_field = 'two_year_recid'

In [ ]:
from statsmodels.stats import proportion

def rationalize(target, data_df, numfields=1):
    def noIntervalOverlap(a1, a2, b1, b2):
        if a1>a2 or b1>b2:
            raise ValueError('unexpected bounds')
        if a2<b1 and a1<b1:
            return True
        if b2<a1 and b1<a1:
            return True
        return False
    breakdown = data_df[trueresult_field]
    full_interval = proportion.proportion_confint(sum(breakdown), len(breakdown), alpha=0.05, method='beta')
    result_df = pd.DataFrame()
    field_groups = [[i] for i in range(len(accessible_fields))]
    for _ in range(numfields-1):
        field_groups = [[f+[i] for i in range(len(accessible_fields)) if i>f[-1]] for f in field_groups]
        field_groups = [e for l in field_groups for e in l]
    for field_ids in field_groups:
        field_names = [accessible_fields[i] for i in field_ids]
        subgroup = data_df
        for field in field_names:
            subgroup = subgroup[subgroup[field] == target[field]]
        subgroup_alt = data_df[~data_df.index.isin(subgroup.index)]
        breakdown = subgroup[trueresult_field]
        breakdown_alt = subgroup_alt[trueresult_field]
        if len(breakdown)>0:
            conf_interval = proportion.proportion_confint(sum(breakdown), len(breakdown), alpha=0.05, method='beta')
            altg_interval = proportion.proportion_confint(sum(breakdown_alt), len(breakdown_alt), alpha=0.05, method='beta')
            result_df = result_df.append({
                'id': target['id'],
                'field_count': len(field_names),
                'fields': field_names,
                'fields_key': (tuple(field_names), tuple([target[fn] for fn in field_names])),
                'sample_size': len(breakdown),
                'percent_recid': sum(breakdown) / len(breakdown),
                'percent_recid_alt': sum(breakdown_alt) / len(breakdown_alt),
                # Confidence interval: in 95% of cases, the true underlying fraction of recidivism will fall within this interval
                # https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval#Clopper%E2%80%93Pearson_interval
                # https://www.statsmodels.org/devel/generated/statsmodels.stats.proportion.proportion_confint.html
                'conf_95_bot': conf_interval[0],
                'conf_95_top': conf_interval[1],
                # Statistical significance
                'significant_baseline': 1 if noIntervalOverlap(conf_interval[0], conf_interval[1], full_interval[0], full_interval[1]) else 0,
                'significant_altgroup': 1 if noIntervalOverlap(conf_interval[0], conf_interval[1], altg_interval[0], altg_interval[1]) else 0,
            }, ignore_index=True)
    return result_df

In [ ]:
breakdown = filter_data_df_src[trueresult_field]
print('true recidivism (training):', sum(breakdown)/len(breakdown))
print('true recidivism (training) range:', proportion.proportion_confint(sum(breakdown), len(breakdown), alpha=0.05, method='beta'))

print()

breakdown = filter_data_df_test[trueresult_field]
print('true recidivism (testing):', sum(breakdown)/len(breakdown))
print('true recidivism (testing) range:', proportion.proportion_confint(sum(breakdown), len(breakdown), alpha=0.05, method='beta'))

In [ ]:
# Mini-example of selecting someone to justify both-sides of
target = filter_data_df[filter_data_df['id'] == 2680].iloc[0] # filter_data_df.sample(n=1).iloc[0]
filter_data_df[filter_data_df['id'] == target['id']]

rationalize_df = pd.DataFrame()
for numfields in [1,2,3]:
    rationalize_df = rationalize_df.append(
        rationalize(target, filter_data_df_src, numfields=numfields),
        ignore_index=True
    )

# rationalize_df.sort_values(['significant_baseline', 'percent_recid'])
rationalize_df[rationalize_df['significant_baseline'] == 1].sort_values('percent_recid')

# Visualize small sample of rationalization bounds

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

count_bothsides = 10
count_factors = 3

def vis(filter_data_df_src, filter_data_df_test, count_bothsides, count_factors):
    plt.rcParams['figure.figsize'] = [15, 5]
    gap_size = 10
    
    # Randomly select someone to justify negative pred for
    for n in range(count_bothsides):
        target = filter_data_df_test[filter_data_df_test[justifying_field] == 0].sample(n=1).iloc[0]
        rationalize_df = pd.DataFrame()
        for numfields in range(1, count_factors+1):
            rationalize_df = rationalize_df.append(
                rationalize(target, filter_data_df_src, numfields=numfields),
                ignore_index=True
            )
        rationalize_df = rationalize_df[rationalize_df['significant_baseline'] == 1]
        dots = plt.scatter(
            [(2*i)+(2*count_factors*n) for i in rationalize_df['field_count']],
            rationalize_df['percent_recid'], 
            alpha=[(i/max(rationalize_df['sample_size']))**0.9 for i in rationalize_df['sample_size']]
        )
        for i in range(1, count_factors+1):
            temp_df = rationalize_df[rationalize_df['field_count']==i]
            range_lo = min(temp_df['conf_95_top'])
            range_hi = max(temp_df['conf_95_bot'])
            pos_x = (2*i)+(2*count_factors*n)+1
            marker = 'o' if (range_lo<=range_hi) else ''
            _ = plt.plot(
                [pos_x,pos_x], 
                [range_lo,range_hi],
                color=dots.get_facecolors()[0][:-1],
                marker=marker,
            )

    # Randomly select someone to justify positive pred for
    for n in range(count_bothsides):
        target = filter_data_df_test[filter_data_df_test[justifying_field] == 1].sample(n=1).iloc[0]
        rationalize_df = pd.DataFrame()
        for numfields in range(1, count_factors+1):
            rationalize_df = rationalize_df.append(
                rationalize(target, filter_data_df_src, numfields=numfields),
                ignore_index=True
            )
        rationalize_df = rationalize_df[rationalize_df['significant_baseline'] == 1]
        dots = plt.scatter(
            [(2*i)+(2*count_factors*n)+(2*count_factors*count_bothsides)+gap_size for i in rationalize_df['field_count']], 
            rationalize_df['percent_recid'], 
            alpha=[(i/max(rationalize_df['sample_size']))**0.9 for i in rationalize_df['sample_size']]
        )
        for i in range(1, count_factors+1):
            temp_df = rationalize_df[rationalize_df['field_count']==i]
            if len(temp_df)==0:
                pass
            if len(temp_df['conf_95_top'])==0:
                print(temp_df)
                raise ValueError()
            range_lo = min(temp_df['conf_95_top'])
            if len(temp_df['conf_95_bot'])==0:
                print(temp_df)
                raise ValueError()
            range_hi = max(temp_df['conf_95_bot'])
            pos_x = (2*i)+(2*count_factors*n)+(2*count_factors*count_bothsides)+gap_size+1
            marker = 'o' if (range_lo<=range_hi) else ''
            _ = plt.plot(
                [pos_x,pos_x], 
                [range_lo,range_hi],
                color=dots.get_facecolors()[0][:-1],
                marker=marker,
            )
    
    # population mean line
    breakdown = filter_data_df_src[trueresult_field]
    src_range = proportion.proportion_confint(sum(breakdown), len(breakdown), alpha=0.05, method='beta')
    plt.axline((1, sum(breakdown)/len(breakdown)), slope=0, alpha=0.4)
    plt.fill_between(
        [0, 4*count_factors*count_bothsides+gap_size+2], 
        [src_range[1], src_range[1]], 
        [src_range[0], src_range[0]], 
        alpha=0.2
    )
    # 50% line
    plt.axline((1, 0.5), slope=0, alpha=0.8)
    plt.show()

In [ ]:
vis(filter_data_df_src, filter_data_df_test, 10, 2)

In [ ]:
vis(filter_data_df_src, filter_data_df_test, 10, 3)

In [ ]:
vis(filter_data_df_src, filter_data_df_test, 20, 2)

In [ ]:
# vis(filter_data_df_src, filter_data_df_test, 20, 3)

In [ ]:
# vis(filter_data_df_src, filter_data_df_test, 20, 4)

# Calculate rationalization for ALL test set samples

In [ ]:
# Calculate rationalization for all test set samples

count_factors = 2

test_rationalizations = pd.DataFrame()
for _, case in filter_data_df_test.iterrows():
    for numfields in range(1, count_factors+1):
        test_rationalizations = pd.concat(
            [
                test_rationalizations,
                rationalize(case, filter_data_df_src, numfields=numfields),
            ],
            ignore_index=True,
        )

# Add column representing the 'side' that each rationalization is in favor of
breakdown = filter_data_df_src[trueresult_field]
justifying_ref_avg = sum(breakdown)/len(breakdown)
test_rationalizations['is_recid_rat_side'] = test_rationalizations.apply(lambda r: 1 if r['percent_recid']>justifying_ref_avg else 0, axis=1)

test_rationalizations[:20]

In [ ]:
# Calculate confidence interval of entire training dataset
breakdown = filter_data_df_src[trueresult_field]
src_range = proportion.proportion_confint(sum(breakdown), len(breakdown), alpha=0.05, method='beta')
src_range

# Visualize all test-set results?

## Show distribution of confidence interval gaps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_bins = 30

# Compute and show confidence interval gaps between "closest to justify 1" vs "closest to justify 0"
temp_gaps = []
for x_id, x_rat in test_rationalizations.groupby(['id']):
    range_lo = min(x_rat['conf_95_top'])
    range_hi = max(x_rat['conf_95_bot'])
    temp_gaps.append(range_hi-range_lo)

_ = plt.hist(temp_gaps, bins=num_bins)
_ = plt.title('all rats')
plt.show()

# CI gaps ... for size-1 sets
temp_gaps = []
for x_id, x_rat in test_rationalizations[test_rationalizations['field_count']==1].groupby(['id']):
    range_lo = min(x_rat['conf_95_top'])
    range_hi = max(x_rat['conf_95_bot'])
    temp_gaps.append(range_hi-range_lo)

_ = plt.hist(temp_gaps, bins=num_bins)
_ = plt.title('rats f=1')
plt.show()

# CI gaps ... for size-2 sets
temp_gaps = []
for x_id, x_rat in test_rationalizations[test_rationalizations['field_count']==2].groupby(['id']):
    range_lo = min(x_rat['conf_95_top'])
    range_hi = max(x_rat['conf_95_bot'])
    temp_gaps.append(range_hi-range_lo)

_ = plt.hist(temp_gaps, bins=num_bins)
_ = plt.title('rats f=2')
plt.show()

# Compute and show confidence interval gaps between "closest to justify 1" vs "closest to justify 0" for ONLY the significant sets (???)
test_signif_rat = test_rationalizations[test_rationalizations['significant_baseline']==1]
temp_gaps = []
for x_id, x_rat in test_signif_rat.groupby(['id']):
    range_lo = min(x_rat['conf_95_top'])
    range_hi = max(x_rat['conf_95_bot'])
    temp_gaps.append(range_hi-range_lo)

_ = plt.hist(temp_gaps, bins=num_bins)
_ = plt.title('iff-significant rats')
plt.show() 

## Show single-rat and double-rat cases

In [ ]:
tags = {'id':[], 'ratted_single':[], 'ratted_double':[]}
for x_id, x_rat in test_rationalizations[test_rationalizations['significant_baseline'] == 1].groupby(['id']):
    tags['id'].append(x_id)
    x_goal = filter_data_df_test[filter_data_df_test['id']==x_id].iloc[0][justifying_field]
    # figure out if goal has been achieved to prediction case-by-case
    tags['ratted_single'].append(any(x_rat['is_recid_rat_side']==x_goal))
    # figure out if goal could be achievable either way case-by-case
    rat_total = sum(x_rat['is_recid_rat_side'])
    tags['ratted_double'].append(rat_total<len(x_rat['is_recid_rat_side']) and rat_total>0)
test_ratsides = pd.DataFrame(tags)
test_ratsides[-10:]

In [ ]:
print('successful rat single', sum(test_ratsides['ratted_single'])/len(test_ratsides))
print('successful rat double', sum(test_ratsides['ratted_double'])/len(test_ratsides))

# Do group multi-element rationalizing

In [ ]:
# Pick out every distinct set of field-value examinations that could be used for justification
fields_keys_gb = test_rationalizations[test_rationalizations['significant_baseline']==1].merge(
    filter_data_df_test[['id', justifying_field]], left_on='id', right_on='id'
).groupby(
    by=['fields_key']
)

fields_keys = pd.DataFrame()
fields_keys['is_recid_rat_side'] = fields_keys_gb.apply(
    lambda x: x['is_recid_rat_side'].mean()
)
fields_keys['is_recid_test_mean'] = fields_keys_gb.apply(
    lambda x: x['is_recid'].mean()
)
fields_keys['samples'] = fields_keys_gb.apply(
    lambda x: len(x)
)
fields_keys['id_samples'] = fields_keys_gb.apply(
    lambda x: list(x['id'])
)
fields_keys[:5]#.sort_values('samples').sort_values('is_recid_test_mean')

In [ ]:
test_rationalizations[:5]

temp_singlemerge = test_rationalizations.merge(filter_data_df_test[['id', justifying_field]], left_on='id', right_on='id')
temp_singlemerge[:5]